<a href="https://colab.research.google.com/github/Shrey-Viradiya/NVIDIA_DALI_PyTorch_keras_tensorflow-Example/blob/main/NVIDIA_DALI_keras_ChestXRAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Feb 28 06:52:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [3]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-tf-plugin-cuda100

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
     |████████████████████████████████| 389.9MB 45kB/s 
Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
     |████████████████████████████████| 317kB 8.4MB/s 
  Created wheel for nvidia-dali-tf-plugin-cuda100: filename=nvidia_dali_tf_plugin_cuda100-0.31.0-cp37-cp37m-linux_x86_64.whl size=99370 sha256=0864a2b3e1634dd9491a4b599605c89fd0cd00520c62dec2500aa789e4738994
  Stored in directory: /root/.cache/pip/wheels/1a/ed/d9/b7adce71d3ed71de92958a4b1a30a7bfe5cd2003e21be04214
Successfully built nvidia-dali-tf-plugin-cuda100


In [4]:
import tensorflow as tf
# from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory

import os
import scipy as sp
import matplotlib.pyplot as plt
import sklearn.metrics
from PIL import Image
import shutil

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types

import nvidia.dali.plugin.tf as dali_tf
import tensorflow as tf
import tensorflow.compat.v1 as tf_v1
import logging

In [5]:
! kaggle datasets download -d tolgadincer/labeled-chest-xray-images

 99% 1.16G/1.17G [00:13<00:00, 117MB/s]
100% 1.17G/1.17G [00:13<00:00, 90.2MB/s]


In [6]:
! unzip -qq -n labeled-chest-xray-images.zip

In [7]:
sets = os.listdir("/content/chest_xray")

In [8]:
train_directory = '/content/chest_xray/train'
valid_directory = '/content/chest_xray/test'

In [9]:
!find '/content/chest_xray/train' -type f -print | wc -l
!find '/content/chest_xray/test' -type f -print | wc -l

5232
624


In [10]:
class Data_Pipeline(Pipeline):
    def __init__(self, batch_size, device, data_path, device_id=0, num_threads=4, seed=0):
        super(Data_Pipeline, self).__init__(
            batch_size, num_threads, device_id, seed)
        self.device = device
        self.reader = ops.FileReader(file_root =data_path, random_shuffle=True)
        self.decode = ops.ImageDecoder(
            device='mixed' if device == 'gpu' else 'cpu',
            output_type=types.RGB)
        self.cmn = ops.CropMirrorNormalize(
            device=device,
            dtype=types.FLOAT,
            std=[255.],
            output_layout="HWC")
        self.rotate = ops.Rotate(device = "gpu")
        self.rng = ops.random.Uniform(range = (-10.0, 10.0))
        self.coin = ops.random.CoinFlip(probability = 0.5)
        self.flip = ops.Flip(device = "gpu")
        self.res = ops.Resize(device="gpu", resize_x=224, resize_y=224, interp_type=types.INTERP_TRIANGULAR)

    def define_graph(self):
        inputs, labels = self.reader(name="Reader")
        images = self.decode(inputs)
        if self.device == 'gpu':
            labels = labels.gpu()
        images = self.cmn(images)
        angle = self.rng()
        images = self.rotate(images, angle=angle)
        images = self.flip(images, horizontal = self.coin(), vertical = self.coin())
        images = self.res(images)
        return (images, labels)

In [11]:
BATCH_SIZE = 4
DROPOUT = 0.2
IMAGE_SIZE = 224
NUM_CLASSES = 20
HIDDEN_SIZE = 512
EPOCHS = 5
DATA_SIZE = 5232
VALIDATION_SIZE = 624
ITERATIONS_PER_EPOCH = DATA_SIZE // BATCH_SIZE
VALIDATION_STEPS = VALIDATION_SIZE // BATCH_SIZE

shapes = (
    (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3),
    (BATCH_SIZE))
dtypes = (
    tf.float32,
    tf.int32)

In [12]:
Pipeline_train = Data_Pipeline(BATCH_SIZE, device='gpu', data_path = train_directory, device_id=0)
Pipeline_valid = Data_Pipeline(BATCH_SIZE, device='gpu', data_path = valid_directory, device_id=0)

In [13]:
with tf.device('/gpu:0'):
    data_set = dali_tf.DALIDataset(
        pipeline=Pipeline_train,
        batch_size=BATCH_SIZE,
        output_shapes=shapes,
        output_dtypes=dtypes,
        device_id=0)
    
    valid_data_set = dali_tf.DALIDataset(
        pipeline=Pipeline_valid,
        batch_size=BATCH_SIZE,
        output_shapes=shapes,
        output_dtypes=dtypes,
        device_id=0)
    
    resnet_model = ResNet50(weights = 'imagenet', include_top=False, input_shape = (224,224,3))

    for layer in resnet_model.layers:
        layer.trainable = False
    
    model = tf.keras.models.Sequential([
                                    resnet_model,
                                    tf.keras.layers.GlobalMaxPooling2D(),
                                    tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
                                    tf.keras.layers.Dropout(DROPOUT),
                                    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

94773248/94765736 [==============================] - 0s 0us/step


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                10260     
Total params: 24,647,060
Trainable params: 1,059,348
Non-trainable params: 23,587,712
_________________________________________________________________


In [15]:
with tf.device('/gpu:0'):
    model.fit(data_set, epochs=EPOCHS, steps_per_epoch=ITERATIONS_PER_EPOCH,  validation_data = valid_data_set, validation_steps=VALIDATION_STEPS)

Epoch 1/5
1308/1308 [==============================] - 41s 24ms/step - loss: 0.7628 - accuracy: 0.7364 - val_loss: 0.4890 - val_accuracy: 0.7388
Epoch 2/5
1308/1308 [==============================] - 31s 24ms/step - loss: 0.4192 - accuracy: 0.7972 - val_loss: 0.5999 - val_accuracy: 0.6923
Epoch 3/5
1308/1308 [==============================] - 32s 24ms/step - loss: 0.3818 - accuracy: 0.8221 - val_loss: 0.5312 - val_accuracy: 0.7131
Epoch 4/5
1308/1308 [==============================] - 32s 24ms/step - loss: 0.3667 - accuracy: 0.8228 - val_loss: 0.3836 - val_accuracy: 0.8253
Epoch 5/5
1308/1308 [==============================] - 32s 24ms/step - loss: 0.3652 - accuracy: 0.8299 - val_loss: 0.4535 - val_accuracy: 0.7692


In [16]:
train_dataset = image_dataset_from_directory(train_directory,
                                             shuffle=True,
                                             labels="inferred",
                                             batch_size=BATCH_SIZE,
                                             image_size=(IMAGE_SIZE, IMAGE_SIZE))

valid_dataset = image_dataset_from_directory(valid_directory,
                                             shuffle=True,
                                             labels="inferred",
                                             batch_size=BATCH_SIZE,
                                             image_size=(IMAGE_SIZE, IMAGE_SIZE))

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Normalization(mean=0., variance=255.),
  tf.keras.layers.experimental.preprocessing.RandomRotation((-10.0, 10.0)),
  tf.keras.layers.experimental.preprocessing.RandomFlip(),
  tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
])

Found 5232 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [17]:
with tf.device('/gpu:0'):
    model.fit(train_dataset, epochs=EPOCHS, validation_data = valid_dataset)

Epoch 1/5
1308/1308 [==============================] - 43s 32ms/step - loss: 1.0010 - accuracy: 0.9310 - val_loss: 0.3705 - val_accuracy: 0.8910
Epoch 2/5
1308/1308 [==============================] - 40s 31ms/step - loss: 0.1217 - accuracy: 0.9622 - val_loss: 0.3439 - val_accuracy: 0.8718
Epoch 3/5
1308/1308 [==============================] - 40s 31ms/step - loss: 0.0956 - accuracy: 0.9667 - val_loss: 0.3082 - val_accuracy: 0.9071
Epoch 4/5
1308/1308 [==============================] - 41s 31ms/step - loss: 0.0798 - accuracy: 0.9700 - val_loss: 0.2409 - val_accuracy: 0.9327
Epoch 5/5
1308/1308 [==============================] - 42s 32ms/step - loss: 0.0861 - accuracy: 0.9694 - val_loss: 0.3569 - val_accuracy: 0.8974
